# Selecting a GPU virtual machine vendor for Machine Learn
There are many competing cloud services providers, such as:
+ [Amazon AWS](https://aws.amazon.com/)
+ [paperspace](https://www.paperspace.com/)
+ [Google Cloud Platform](https://cloud.google.com/)

Really great comparison from GPU computation perspective has been published recently on 
[**Maximize your GPU dollars**](https://towardsdatascience.com/maximize-your-gpu-dollars-a9133f4e546a), which I strongly recommend for in depth analysis. THe general conclusion is that for stand alone virtual machine both AWS and GCP offer similar prices. 
For example CPU instance (2 vCPU, 7.5 GB RAM, 50 GB HDD) - the price is 5 USD for 1 month assuming 1h per day usage (0.16 USD per hour). On the other hand GPU instance with better specification (4 vCPU, 15 GB RAM, 50 GB HDD, GPU Tesla K80) - the price is 21 USD per month (0.66 USD per hour). But when higher computing power of GPU is necessary, the more powerful GPU can be selected such as Tesla P100, then the price rises to 50 USD per month (1.7 USD per hour). Unfortunately, for the CPU and GPU you are being charged per hour, but storage (HDD or SDD) are being charged regardless if the machine is working or stopped. More exact estimations can be found on GCP [calculator](https://cloud.google.com/products/calculator/).

Different model of instance canbe found on [paperspace](https://www.paperspace.com/). Beside typical virtual machines offerd as a Core product, papaerspace also offers something called **Gradient**. Gradient offers only an access to instance over jupyter notebooks. The advantage is limited configuration (almost to zero, it is ready to work just in 3 minutes), and you are charged only for the time you use the instance. No additional costs for, when the instance is stopped. The prices are affordable, and CPU instance starts from 0.05 USD per hour (1 vCPU, 2 GB RAM, 250 SSD). The GPU enabled machines stars from 0.6 USD per hour (Tesla K80, 2 vCPU, 12 GB RAM, 250 GB SSD), and 1.7 USD per hour for better machine (Tesla P100, 4 vCPU, 24 GB RAM, 250 GB SSD).
The disadvantage of such solution is limited possibilities of installing additional packages, other means of access than through Jupyter Notebook, or changing instance type between CPU and GPU.



According to conclusion in mentioned article, and my own experience with AWS, paperspace, and Google Cloud Platform (GCP for short), my choice was on GCP service. There are few reasons of such decision, just to mention few:
* competitive prices
* simpler maintains than in AWS
* 300$ for new users to spent during 1 year in GCP!
* possibility of modifying instances

I would like to comment the last point. My wish was to find a solution, where I would be able to create an inexpensive virtual machine for general purpose (low price with simple, few CPU machines, price below 0.1$ per hour). But the most important point is, that from time to time I would like to have GPU computing capabilities, for especially for Deep Learning. With GCP (as well as in AWS) I can create a cheap instance, where I would make regular work, and code development, and when GPU would be necessary, all I need is to stop instance, and then add GPU to its configuration. This way you can have a flexible solution with GPU, for which you will pay only on time slots when you really need it. 

## VM instance creation
Here I will assume that you already have a GCP account, if not please find any tutorial on web or GCP website.

First we need to open Console of GCP and then **Compute Engine**. Then open "VM instances" - where you can add, remove or edit your virtual machines.

1. First click create virtual machine button:
<img class="img-center" src="../../images/vnc/create01.PNG" alt="VM_bar" style="width: 400px"/>
2. Then  there are multiple choices you can make to your machine: name, number of CPU, adding GPU card (I do not advise adding GPU now, it can be added later), image from which VM should be created. For now I suggest the following choices:
Region: us-central(Iowa), CPU: 4, RAM 15GB, Image **Deep Learning Image: PyTorch 0.4.1...**, increasing storage form 30GB into 50GB. The cost of such configuration is just $0.15 per hour.

Once you have all necessary parameters specified, clicked create button.
My configuration looked as follows:
<img class="img-center" src="../../images/vnc/create02.PNG" alt="VM_config" style="width: 400px"/>

If creation of instance was successful, you should see newly created machine in your VM listing:
<img class="img-center" src="../../images/vnc/create03.PNG" alt="VM_list" style="width: 400px"/>
Congratulations, you have your own Virtual Machine somewhere in the cloud.

## VNC connection to VM in cloud

In current state, the most convenient way of accessing your VM is through web based SSH connection provided by GCP, all you need to do is just click on **SHH** name by your instance. 
Then new window will open, and after some time you will see prompt line of your instance:
<img class="img-center" src="../../images/vnc/prompt01.PNG" alt="prompt01" style="width: 400px"/>
You can wander if your instance is not protected since you did not provide any password - actually it is secured quite well, and login was done over SSH channel utilizing embedded SSH-RSA key. Later I will show you how you can login to VM using *putty*.

Ok, in order to have fully functional VM, which would give you a feeling of real computer (only remote) we need remote access to its GUI interface. Here I will describe how to access your VM with I guess most popular VNC client.

Unfortunately, it is not just straight forward, since there is some packages that needs to be installed, and some configurations has to be created.

## VNC server installation on VM

I was inspired by other [article](https://www.digitalocean.com/community/tutorials/how-to-install-and-configure-vnc-on-debian-9) describing VNC connection on Debian systems.

In the SSH console please input following commands, in order to install Xfce desktop environment on our VM:

```python
sudo apt update
sudo apt install xfce4 xfce4-goodies
```
During installation select English, then Enter key.

In case of problem during installation, try:
```python
ps aux | grep -i apt
sudo kill <process id> or
sudo dpkg --configure -a
```

After installation of Xfce desktop, it is time to install VNC server:

```python
sudo apt install vnc4server
```

After successful installation we need to run  *vncserver* in order to finish its settings:
```python
sudo vncserver
```
Here you will be asked to provide password for remote accessing VNC service (though VNC viewer on your local computer). Then repeat password, and refuse creation on password for View only 
```python
ptwnuk@myinstance03:~$ sudo vncserver
You will require a password to access your desktops.
Password:
Verify:
Would you like to enter a view-only password (y/n)? n
/usr/bin/xauth:  file /root/.Xauthority does not exist
New 'myinstance03.europe-west4-a.c.white-collector-222018.internal:1 (root)' desktop at :1 on machine myinstance03.
europe-west4-a.c.white-collector-222018.internal
Starting applications specified in /etc/X11/Xvnc-session
Log file is /root/.vnc/myinstance03.europe-west4-a.c.white-collector-222018.internal:1.log
Use xtigervncviewer -SecurityTypes VncAuth -passwd /root/.vnc/passwd :1 to connect to the VNC server.
```

Now you can check that VNC server is already  running on your VNC:
Then you can list VNC sessions using:
```python
vncserver -list
```

Now we need to change VNC configuration file, in order to make sure, that whenever VNC is stated it should execute Xfce desktop.
But before editing file, we must kill VNC process, which stared during our initialization procedure above. For that type the following:

```python
ptwnuk@myistance:~$ vncserver -kill :1
Killing Xtigervnc process ID 18595... success!
```

If there is already configuration file for VNC server then lets make a copy of it:
```python
mv ~/.vnc/xstartup ~/.vnc/xstartup.bak
nano ~/.vnc/xstartup
```

Then inside the file past:
```python
~/.vnc/xstartup
#!/bin/bash
xrdb $HOME/.Xresources
startxfce4 &
```
Then press Ctrl+X, then confirm saving file.
```python
sudo chmod +x ~/.vnc/xstartup
vncserver
```
Now your VNC server should be running! The only missing component is connection from our local VNC viewer computer to remote VNC server on VM.

For that we will use *putty* connection over SSH, with VNC ports tunneling. First we need to get access to VM on GCP with *putty*.

## Connection to VM over SSH with *putty*

Beside convenient control of VM through command line provided in Internet browser (by clicking on SSH label by VM name), more capabilities are possible when accessing through *putty*, when for example port forwarding can be enabled.

First of all, we need to generate a private and public SSH-RSA keys. For that you can use "puttygen", you can download it (as well as putty) [here](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html).

In *puttygen* select key type RSA, and in the filed "Key comment" input your VM identity (you can check it over console with command *whoami*). It should look like in the image below:
<img class="img-center" src="../../images/vnc/putty01.PNG" style="width: 400px"/>

Now you should save your private key (ppk file, you can additionally protect it with additional password for even better security) - which later on will be necessary in putty authentication configuration.
Moreover, you can save your public key - which will have to be provided in GCP in SSH keys pool.

Your public key should have the following format:
*ssh-rsa SOME_RANDOM_CHARACTERS== your_Id* 
As a note, saved public key with puttygen includes some line braking (end lines) which makes it hard to provide directly into GCP ssh-key vault. For that reason the most convenient way is by copy-paste method, so with mouse just select in *puttygen* a output of Public key window (just remember to select all text, it does not fit into output window).

Now we have to provide our public key into GCP. Here it can be done over two means: either apply it to general setting, so all VM machines will utilize this key, or it can be done on each VM separately. For convenient use, I prefer the first method, which will be described below. 

First you need open Compute Engine/Metdata/SSH Keys, and click edit button, and there you will be able to add new public key. Now just click "add item" and in the new box paste a public key which was generated by puttygen.
Your SSH should look similar as image below:
<img class="img-center" src="../../images/vnc/putty02.PNG" style="width: 400px"/>
Now just remember to click save button, so public key will be stored permanently.

Now we need to configure our putty for accessing VM.
First configure private key, for that go to Connections/SSH/Auth, and Authentication parameters click Brows button and select your previously saved private key.
<img class="img-center" src="../../images/vnc/putty03.PNG" style="width: 400px"/>

Next go to Sessions and in Host Name provide a public IP of your VM. In case of not static IP, each time you start VM your public IP can be different - you can read it in your VM console. 
<img class="img-center" src="../../images/vnc/putty04.PNG" style="width: 400px"/>
Next provide some friendly name to your session, for example "my_Cloud_instance", and click save button.

Properly configured putty should look like that:
<img class="img-center" src="../../images/vnc/putty05.PNG" style="width: 400px"/>
Now just click Open button. In case of connection to new IP, you can get a prompt for accepting new key - just click ok.

New black putty window should appear, where you will be prompted: *login as*, here you need one more time provide your account name, as in the case of generating private/public keys.

If everything worked correctly, you should get access to your VM command line over SSH with putty:
<img class="img-center" src="../../images/vnc/putty06.PNG" style="width: 400px"/>

### VNC port tunnelling
Although now we have an access to VM with putty, we still can't reach vncserver on our VM. 

For that VNC ports have to be tunnelled over SSH, which can be configured in putty. First close your previous SSH connection, and open putty configuration window. Then load your previously saved session, so you can edit it. Proper configuration is shown on image below:
<img class="img-center" src="../../images/vnc/putty07.PNG" style="width: 400px"/>
then click add button, and one more time save updated session.
Now you can open connection one more time to your VM.
The only step we need to make is to run vncserver on VM, so in command line just type:
```python 
vncserver```

### Connection with VNC Viewer to VM
First you need to install VNC viewer, which you can find [here](https://www.realvnc.com/en/connect/download/viewer/).

In VNC Viewer click File/New Connection - where you can specify new connection. In the filed VNC server provide: loacalhost:5901 - this address over SSH tunneling should reach our VNC server. Then add any name for that newly created connection and click Ok button. 
Now just double click on new connection - to open vnc connection to your remote computer. Authentication window will appear, where you need provide a password which you choose during configuration of vncserver on remote computer. You can select "Remember password" for future connections.

If everything was configured properly, you should already see desktop on your remote virtual machine - congratulations!

In some cases, colors could look unnatural, this is due to connection speed optimization. If your connection is of high quality and your both machines are fast enough, you can improve your color rendering by modifying connection parameters in VNC Viewer. Just click with right button on the title top bar of your VNC connection window, and then select Properties. Go to Expert settings, and modify ColorLevel property with the following value "pal8" or "full" depending on your connection, and then click Ok.
<img class="img-center" src="../../images/vnc/putty08.PNG" style="width: 400px"/>

After correction of color schema, your desktop should look similar to this:
<img class="img-center" src="../../images/vnc/putty09.PNG" style="width: 400px"/>
Now you can fully enjoy a reliable connection to your virtual machine desktop :)

## Some customization
To fully enjoy your remote desktop, you should install some web browser, since in current state no browser is installed. I installed a firefox browser with the following command:

```python
sudo apt install firefox-esr
```

The other useful configuration is changing default text editor from emacs into Mousepad.
You can do it by creating empty text file (by clicking right button on desktop Create Document/Empty File). Then click right button on that document and then Open With/Open With Other Application and then select Mousepad and additionally check box: Use as a default.